In [1]:
%%capture --no-stderr
%pip install -U langsmith ragas numpy openai

In [2]:
import langsmith

client = langsmith.Client()
dataset_url = (
    "https://smith.langchain.com/public/56fe54cd-b7d7-4d3b-aaa0-88d7a2d30931/d"
)
dataset_name = "BaseCamp Q&A"
client.clone_public_dataset(dataset_url)

Dataset(name='BaseCamp Q&A', description='Taken from: https://basecamp.com/handbook', data_type=<DataType.kv: 'kv'>, id=UUID('ae36b34b-e912-4273-a7df-a53b4bd8a3af'), created_at=datetime.datetime(2024, 10, 30, 15, 23, 9, 512309, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 10, 30, 15, 23, 9, 512309, tzinfo=datetime.timezone.utc), example_count=0, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None)

## Download all the sources

In [3]:
import io
import os
import zipfile

import requests

# Fetch the source documents
url = "https://storage.googleapis.com/benchmarks-artifacts/basecamp-data/basecamp-data.zip"

response = requests.get(url)


with io.BytesIO(response.content) as zipped_file:
    with zipfile.ZipFile(zipped_file, "r") as zip_ref:
        zip_ref.extractall()

data_dir = os.path.join(os.getcwd(), "data")
docs = []
for filename in os.listdir(data_dir):
    if filename.endswith(".md"):
        with open(os.path.join(data_dir, filename), "r") as file:
            docs.append({"file": filename, "content": file.read()})

## Simple InMemory Vector Store

In [4]:
from typing import List

import numpy as np
import openai
from langsmith import traceable


class VectorStoreRetriever:
    def __init__(self, docs: list, vectors: list, oai_client):
        self._arr = np.array(vectors)
        self._docs = docs
        self._client = oai_client

    @classmethod
    async def from_docs(cls, docs, oai_client):
        embeddings = await oai_client.embeddings.create(
            model="text-embedding-3-small", input=[doc["content"] for doc in docs]
        )
        vectors = [emb.embedding for emb in embeddings.data]
        return cls(docs, vectors, oai_client)

    @traceable
    async def query(self, query: str, k: int = 5) -> List[dict]:
        embed = await self._client.embeddings.create(
            model="text-embedding-3-small", input=[query]
        )
        # "@" is just a matrix multiplication in python
        scores = np.array(embed.data[0].embedding) @ self._arr.T
        top_k_idx = np.argpartition(scores, -k)[-k:]
        top_k_idx_sorted = top_k_idx[np.argsort(-scores[top_k_idx])]
        return [
            {**self._docs[idx], "similarity": scores[idx]} for idx in top_k_idx_sorted
        ]

## Naive Rag

In [5]:
from langsmith import traceable
from langsmith.wrappers import wrap_openai


class NaiveRagBot:
    def __init__(self, retriever, model: str = "gpt-4-turbo-preview"):
        self._retriever = retriever
        # Wrapping the client instruments the LLM
        # and is completely optional
        self._client = wrap_openai(openai.AsyncClient())
        self._model = model

    @traceable
    async def get_answer(self, question: str):
        similar = await self._retriever.query(question)
        response = await self._client.chat.completions.create(
            model=self._model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful AI assistant."
                    " Use the following docs to help answer the user's question.\n\n"
                    f"## Docs\n\n{similar}",
                },
                {"role": "user", "content": question},
            ],
        )

        # The RAGAS evaluators expect the "answer" and "contexts"
        # keys to work properly. If your pipeline does not return these values,
        # you should wrap in a function that provides them.
        return {
            "answer": response.choices[0].message.content,
            "contexts": [str(doc) for doc in similar],
        }

In [6]:
retriever = await VectorStoreRetriever.from_docs(docs, openai.AsyncClient())
rag_bot = NaiveRagBot(retriever)

In [7]:
response = await rag_bot.get_answer("How much time off do we get?")
response["answer"][:150]

'You get several types of time off at 37signals, including:\n\n### Paid Time Off (PTO)\n- **Vacation Time:** 37signals offers 18 days of paid time off plu'

## Evaluate

In [10]:
%pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 32.0 MB/s eta 0:00:00
  Using cached click-8.1.7-py3-none-any.whl (97 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain.smith import RunEvalConfig
from ragas.integrations.langchain import EvaluatorChain
from ragas.metrics import (
    answer_correctness,
    answer_relevancy,
    context_precision,
    context_recall,
    faithfulness,
)

# Wrap the RAGAS metrics to use in LangChain
evaluators = [
    EvaluatorChain(metric)
    for metric in [
        answer_correctness,
        answer_relevancy,
        context_precision,
        context_recall,
        faithfulness,
    ]
]
eval_config = RunEvalConfig(custom_evaluators=evaluators)

In [13]:
results = await client.arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=rag_bot.get_answer,
    evaluation=eval_config,
)

View the evaluation results for project 'extraneous-window-46' at:
https://smith.langchain.com/o/bc1abc05-6850-4a1a-8627-3b31ec53e09f/datasets/ae36b34b-e912-4273-a7df-a53b4bd8a3af/compare?selectedSessions=c45a0371-8bc6-4755-a027-9a4fe4cf80b3

View all tests for Dataset BaseCamp Q&A at:
https://smith.langchain.com/o/bc1abc05-6850-4a1a-8627-3b31ec53e09f/datasets/ae36b34b-e912-4273-a7df-a53b4bd8a3af
[>                                                 ] 0/21

Error evaluating run 770493b4-8d2b-4a52-8c84-4e3b6af1ebc3 with EvaluatorChain: RuntimeError("There is no current event loop in thread 'asyncio_1'.")
Traceback (most recent call last):
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain_core/tracers/evaluation.py", line 128, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/ragas/integrations/langchain.py", line 198, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 170, in invoke
    raise e
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 160, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/

[->                                                ] 1/21

Error evaluating run 03862567-77d7-4567-b7b4-9829d817d6a7 with EvaluatorChain: RuntimeError("There is no current event loop in thread 'asyncio_4'.")
Traceback (most recent call last):
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain_core/tracers/evaluation.py", line 128, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/ragas/integrations/langchain.py", line 198, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 170, in invoke
    raise e
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 160, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/

[---->                                             ] 2/21

...

[-------------------------------------------->     ] 19/21

Error evaluating run 2b0c2fe1-8d26-44b7-8f13-246041e72194 with EvaluatorChain: RuntimeError("There is no current event loop in thread 'asyncio_2'.")
Traceback (most recent call last):
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain_core/tracers/evaluation.py", line 128, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/ragas/integrations/langchain.py", line 198, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 170, in invoke
    raise e
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 160, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/

[----------------------------------------------->  ] 20/21

Error evaluating run aa5d7e42-0313-483c-aa8f-541ad6cf8409 with EvaluatorChain: RuntimeError("There is no current event loop in thread 'asyncio_2'.")
Traceback (most recent call last):
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain_core/tracers/evaluation.py", line 128, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/ragas/integrations/langchain.py", line 198, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 170, in invoke
    raise e
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/site-packages/langchain/chains/base.py", line 160, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/Users/harsha/workspace/pgnd-ai/model-trys/.venv/lib/python3.10/

[------------------------------------------------->] 21/21